Primer paso es leer BD del servidor donde se tenga la tabla de los pesometros antiguos con problemas.
En este caso es el servidor de google,
Primero vamos a ver si tenemos conexion al servidor , mostrando las tablas que tiene la bd

In [2]:
print("holaaa")

holaaa


In [3]:
import mysql.connector
import time
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter


db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
consulta="SHOW TABLES"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print (" termino la consulta")



conexion a gc
camiones_reg
cargador_reg
pesometroreparado2
pesometroreparado3
pesometros
 termino la consulta


Los datos en bruto estan en la tabla pesometro columna acc1 pesometro integral y acc2 pesometro grava 3/4.
row[1]=fecha, row[5]=integral, row[6]=grava3/4.>COn los siguiente vamos a describir la tabla

In [4]:
consulta="DESCRIBE pesometros"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print ("se termino la descripcion de pesometros ")


id
fecha
tph1
tph2
tph3
acc1
acc2
acc3
diff1
diff2
diff3
se termino la descripcion de pesometros 


vamos a leer el primer y ultimo registro de la tabla pesometro que queremo insertar en la tabla reparada.
es impportante esta consulta, ya que de aca se obtiene el primer registro acumulado para reststar al insertar 
en la nuev tabla,


In [5]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")

integralx=[]
gravax=[]
fechaIni="'2019-08-11 02:15:38'"
fechaFin="'2019-08-16 22:47:00'" 

#consula
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha >= "+fechaIni+ "ORDER BY id ASC LIMIT 1"
print(consulta)
cur.execute(consulta)
print ("El primer registro de la consulta ")
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")
print ("El ultimo registro de la consulta aaaaa")
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha <= "+fechaFin+ "ORDER BY id DESC LIMIT 1"
cur.execute(consulta)
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")
print(integralx)
print(gravax)
restaint=integralx[0]
restagrav=gravax[0]



conexion a gc
SELECT * FROM pesometros WHERE fecha >= '2019-08-11 02:15:38'ORDER BY id ASC LIMIT 1
El primer registro de la consulta 
54218882 2019-08-11 02:15:41 260 100
 
El ultimo registro de la consulta aaaaa
54314731 2019-08-16 22:45:07 0 0
 
[256732994400, 265377798100]
[84825164500, 87408469800]


Ahora vamos a reparar los datos


In [57]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
#consula
tabla="pesometros"

integral=[]
grava=[]
restaint=integralx[0]
restagrav=gravax[0]

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)



conexion a gc
[254724709600, 265377798100]
[84099887700, 87408469800]


In [ ]:
#de atras deben venir los arreglos 
#restaint=integralx[0]
#restagrav=gravax[0]
#que son los valores acumulados, los cuales vamos a venecitar para restar
integral=[]
grava=[]
restaint=integralx[0]
restagrav=gravax[0]
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("vamos a comenzar la revision e insercion")
consulta="select * FROM "+tabla+" WHERE fecha between "+fechaIni+" and "+fechaFin+" "  
print(consulta)
cur.execute(consulta)

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)


for row in cur.fetchall():
     #print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
    if contador==0:
         anteriorint=row[5]
         anteriorgrav=row[6]
         anteriorintsql=0
         anteriorgravasql=0
    if row[5]-anteriorint>3000:

        print("entro aca")
        restaint=restaint+row[5]-anteriorint
        print ("cambio de Integral")
    if row[6]-anteriorgrav>3000:
        restagrav=restagrav+row[6]-anteriorgrav
              
   
    print(str(contador)+"  "+str(row[1])+"   "+str(row[5])+"   "+str(row[5]-restaint)+"---------"+str(row[6])+"   "+str(row[6]-restagrav))
    integral.append(row[5]-restaint)
    grava.append(row[6]-restagrav)
    #aca debo hacer el insert a la nueva tabla
    consulta="INSERT INTO `pesometroreparado3` (`hora`, `acc1`, `acc2`, `acc3`)VALUES ('"+str(row[1])+"', '"+str(row[5]-restaint-anteriorintsql)+"', '"+str(row[6]-restagrav-anteriorgravasql)+"', NULL);"
    #print(consulta)
    #print(anteriorint)
    cur.execute(consulta)
    db.commit()
    anteriorint=row[5]
    anteriorgrav=row[6]
    anteriorintsql=row[5]-restaint
    anteriorgravasql=row[6]-restagrav
    contador=contador+1

print("la produccion de integral es : ")
print(str(integral[len(integral)-1]))
print("la produccion de integral es en toneladas: ")
print(str(integral[len(integral)-1]*100/1000))

print("la produccion de grava es : ")
print(str(grava[len(grava)-1]))

print("la produccion de grava en toneladas es : ")
print(str(grava[len(grava)-1]*100/1000))

vamos a comenzar la revision e insercion
select * FROM pesometros WHERE fecha between '2019-08-11 02:15:38' and '2019-08-16 22:47:00' 
[256732994400, 265377798100]
[84825164500, 87408469800]
0  2019-08-11 02:15:41   256732994400   0---------84825164500   0
1  2019-08-11 02:15:44   256732994600   200---------84825164600   100
2  2019-08-11 02:15:47   256732994800   400---------84825164700   200
3  2019-08-11 02:15:50   256732995000   600---------84825164800   300
4  2019-08-11 02:15:53   256732995300   900---------84825164900   400
5  2019-08-11 02:15:56   256732995500   1100---------84825165000   500
6  2019-08-11 02:15:59   256732995800   1400---------84825165100   600
7  2019-08-11 02:16:02   256732996000   1600---------84825165100   600
8  2019-08-11 02:16:05   256732996300   1900---------84825165200   700
9  2019-08-11 02:16:08   256732996500   2100---------84825165300   800
10  2019-08-11 02:16:11   256732996700   2300---------84825165400   900
11  2019-08-11 02:16:14   2567329969

109  2019-08-11 02:21:22   256742191700   27000---------84828868400   13700
110  2019-08-11 02:21:25   256742191900   27200---------84828868500   13800
111  2019-08-11 02:21:28   256742192100   27400---------84828868600   13900
112  2019-08-11 02:21:31   256742192400   27700---------84828868700   14000
113  2019-08-11 02:21:34   256742192600   27900---------84828868800   14100
114  2019-08-11 02:21:37   256742192900   28200---------84828868900   14200
115  2019-08-11 02:21:40   256742193100   28400---------84828869000   14300
116  2019-08-11 02:21:44   256742193400   28700---------84828869100   14400
117  2019-08-11 02:21:47   256742193600   28900---------84828869200   14500
118  2019-08-11 02:21:50   256742193800   29100---------84828869300   14600
119  2019-08-11 02:21:53   256742194100   29400---------84828869400   14700
120  2019-08-11 02:21:56   256742194400   29700---------84828869500   14800
121  2019-08-11 02:21:59   256742194600   29900---------84828869600   14900
122  2019-08

217  2019-08-11 02:26:59   256761415500   50300---------84832588200   26300
218  2019-08-11 02:27:02   256761415700   50500---------84832588300   26400
219  2019-08-11 02:27:05   256761415900   50700---------84832588500   26600
220  2019-08-11 02:27:08   256761416100   50900---------84832588600   26700
221  2019-08-11 02:27:11   256761416300   51100---------84832588700   26800
222  2019-08-11 02:27:14   256761416600   51400---------84832588900   27000
223  2019-08-11 02:27:17   256761416800   51600---------84832589000   27100
224  2019-08-11 02:27:20   256761417000   51800---------84832589200   27300
225  2019-08-11 02:27:23   256761417300   52100---------84832589300   27400
226  2019-08-11 02:27:26   256761417500   52300---------84832589400   27500
227  2019-08-11 02:27:30   256761417800   52600---------84832589600   27700
228  2019-08-11 02:27:33   256761418000   52800---------84832589700   27800
229  2019-08-11 02:27:36   256761418200   53000---------84832589800   27900
230  2019-08

326  2019-08-11 02:32:35   256761440300   75100---------84840020100   38800
327  2019-08-11 02:32:38   256761440500   75300---------84840020200   38900
328  2019-08-11 02:32:41   256761440800   75600---------84840020300   39000
329  2019-08-11 02:32:44   256761441000   75800---------84840020400   39100
330  2019-08-11 02:32:47   256761441300   76100---------84840020500   39200
331  2019-08-11 02:32:51   256761441500   76300---------84840020600   39300
332  2019-08-11 02:32:54   256761441700   76500---------84840020700   39400
333  2019-08-11 02:32:57   256761441900   76700---------84840020800   39500
334  2019-08-11 02:33:00   256761442200   77000---------84840020900   39600
335  2019-08-11 02:33:03   256761442400   77200---------84840021000   39700
336  2019-08-11 02:33:06   256761442700   77500---------84840021100   39800
337  2019-08-11 02:33:09   256761442900   77700---------84840021200   39900
338  2019-08-11 02:33:12   256761443100   77900---------84840021400   40100
339  2019-08

434  2019-08-11 02:38:10   256770460900   95600---------84840032900   51600
435  2019-08-11 02:38:13   256770461200   95900---------84840033000   51700
436  2019-08-11 02:38:16   256770461400   96100---------84840033200   51900
437  2019-08-11 02:38:19   256770461600   96300---------84840033300   52000
438  2019-08-11 02:38:22   256770461900   96600---------84840033500   52200
439  2019-08-11 02:38:25   256770462100   96800---------84840033600   52300
440  2019-08-11 02:38:28   256770462400   97100---------84840033700   52400
441  2019-08-11 02:38:31   256770462700   97400---------84840033900   52600
442  2019-08-11 02:38:35   256770463000   97700---------84840034000   52700
443  2019-08-11 02:38:38   256770463300   98000---------84840034200   52900
444  2019-08-11 02:38:41   256770463500   98200---------84840034300   53000
445  2019-08-11 02:38:44   256770463800   98500---------84840034400   53100
446  2019-08-11 02:38:47   256770464100   98800---------84840034600   53300
447  2019-08

...

14328  2019-08-12 01:35:15   258547658000   2887500---------85339665500   1247000
14329  2019-08-12 01:35:18   258547658300   2887800---------85339665600   1247100
14330  2019-08-12 01:35:21   258547658600   2888100---------85339665700   1247200
14331  2019-08-12 01:35:24   258547658800   2888300---------85339665800   1247300
14332  2019-08-12 01:35:27   258547659100   2888600---------85339665900   1247400
14333  2019-08-12 01:35:31   258547659400   2888900---------85339666000   1247500
14334  2019-08-12 01:35:34   258547659600   2889100---------85339666100   1247600
14335  2019-08-12 01:35:37   258547659900   2889400---------85339666200   1247700
14336  2019-08-12 01:35:40   258547660300   2889800---------85339666300   1247800
14337  2019-08-12 01:35:43   258547660500   2890000---------85339666400   1247900
14338  2019-08-12 01:35:46   258547660800   2890300---------85339666600   1248100
14339  2019-08-12 01:35:49   258547661000   2890500---------85339666700   1248200
14340  2019-08-1

14429  2019-08-12 01:40:33   258547674600   2904100---------85344477900   1259200
14430  2019-08-12 01:40:36   258547674600   2904100---------85344478000   1259300
14431  2019-08-12 01:40:39   258547674600   2904100---------85344478200   1259500
14432  2019-08-12 01:40:42   258547674600   2904100---------85344478300   1259600
14433  2019-08-12 01:40:45   258547674600   2904100---------85344478400   1259700
14434  2019-08-12 01:40:48   258547674600   2904100---------85344478500   1259800
14435  2019-08-12 01:40:51   258547674600   2904100---------85344478700   1260000
14436  2019-08-12 01:40:54   258547674600   2904100---------85344478800   1260100
14437  2019-08-12 01:40:57   258547674600   2904100---------85344478900   1260200
14438  2019-08-12 01:41:00   258547674600   2904100---------85344479100   1260400
14439  2019-08-12 01:41:03   258547674600   2904100---------85344479200   1260500
14440  2019-08-12 01:41:06   258547674600   2904100---------85344479300   1260600
14441  2019-08-1

14529  2019-08-12 01:45:43   258559691100   2920500---------85349322700   1268800
14530  2019-08-12 01:45:46   258559691400   2920800---------85349322700   1268800
14531  2019-08-12 01:45:50   258559691700   2921100---------85349322800   1268900
14532  2019-08-12 01:45:53   258559692100   2921500---------85349322800   1268900
14533  2019-08-12 01:45:56   258559692500   2921900---------85349322800   1268900
14534  2019-08-12 01:45:59   258559692800   2922200---------85349322800   1268900
14535  2019-08-12 01:46:02   258559693200   2922600---------85349322800   1268900
14536  2019-08-12 01:46:05   258559693500   2922900---------85349322800   1268900
14537  2019-08-12 01:46:08   258559693900   2923300---------85349322800   1268900
14538  2019-08-12 01:46:11   258559694200   2923600---------85349322800   1268900
14539  2019-08-12 01:46:14   258559694500   2923900---------85349322800   1268900
14540  2019-08-12 01:46:17   258559694900   2924300---------85349322900   1269000
14541  2019-08-1

14630  2019-08-12 01:50:56   258571544500   2952500---------85349326700   1272800
14631  2019-08-12 01:50:59   258571544800   2952800---------85349326800   1272900
14632  2019-08-12 01:51:02   258571545100   2953100---------85349327000   1273100
14633  2019-08-12 01:51:05   258571545400   2953400---------85349327100   1273200
14634  2019-08-12 01:51:08   258571545700   2953700---------85349327200   1273300
14635  2019-08-12 01:51:11   258571546000   2954000---------85349327400   1273500
14636  2019-08-12 01:51:14   258571546300   2954300---------85349327500   1273600
14637  2019-08-12 01:51:17   258571546600   2954600---------85349327600   1273700
14638  2019-08-12 01:51:20   258571546900   2954900---------85349327800   1273900
14639  2019-08-12 01:51:23   258571547200   2955200---------85349327900   1274000
14640  2019-08-12 01:51:27   258571547500   2955500---------85349328000   1274100
14641  2019-08-12 01:51:30   258571547700   2955700---------85349328200   1274300
14642  2019-08-1

14728  2019-08-12 03:01:38   258678736600   2960800---------85384239000   1280800
14729  2019-08-12 03:01:41   258678737000   2961200---------85384239100   1280900
14730  2019-08-12 03:01:44   258678737100   2961300---------85384239200   1281000
14731  2019-08-12 03:01:47   258678737300   2961500---------85384239300   1281100
14732  2019-08-12 03:01:50   258678737500   2961700---------85384239400   1281200
14733  2019-08-12 03:01:53   258678737800   2962000---------85384239500   1281300
14734  2019-08-12 03:01:56   258678738100   2962300---------85384239600   1281400
14735  2019-08-12 03:02:00   258678738400   2962600---------85384239600   1281400
14736  2019-08-12 03:02:03   258678738400   2962600---------85384239700   1281500
14737  2019-08-12 03:02:09   258678738400   2962600---------85384239800   1281600
14738  2019-08-12 03:02:21   258678738400   2962600---------85384239900   1281700
14739  2019-08-12 03:02:24   258678738600   2962800---------85384239900   1281700
14740  2019-08-1